# API需求


1. Generate Example numpy Data example_np: in size of batch x m x n x t x c, output is a float number (range from 0 to 3000)
{randomly get value from -1 to 1}

Example: 
function generate_example_np():
 return numpy array
example_np=generate_example_np()

 
 
2. 我有一个非常大的数据集，in size of q x m x n x t x c. q代表样本数量，帮我构思一种最有效率的存储改数据集的方式。
Generate Example numpy Data {randomly get value from -1 to 1} 并且写入中
save_dir=r"C:\Datasets\Zhejiang20-23RS\temp_training"

3. 
Make a function generate_loader()
{randomly split training and test data of 80% - 20%}

Example: 
if data is numpy array function generate_loader(example_np):
 return: (lightning dataloader) train_loader, test_loader

 if input is string, 给定save_dir， 读取数据

 然后帮我写一个测试代码

 
3. Make a regression model by using lightning.
input: batch x c x w x h x t   
output: a float number

class SpatioTemporalModel
 ST_attention 
 backbone=resnet18 feature
 
 
 output=st(input) 
 output=backbone(output) 
 then make output be a float number
 
class ST_attention：
    spatial_attention  # hxwx1
    temporal_attention # 1x1xt
    st[hxwxt]=sigmoid(spatial_attention x (elementwise multiply) temporal_attention) 

    output=st(input)


Help me annotate them in style of Google coding python doc string WITH type hinting





In [3]:
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoGeo")
from tqdm import tqdm 
from sample.unit_test import generate_example, generate_large_dataset	
from sample.dataset import generate_datasets

In [4]:

# Test 1
q, m, n, t, c = 5000, 10, 20, 20, 20  # Example dimensions
test_data_0d = generate_example((q, c))
test_data_1d = generate_example((q, t, c))
test_data_2d = generate_example((q, m, n, c))
test_data_3d = generate_example((q, m, n, t, c))

# Example to get data in different formats
numpy_data, numpy_labels = generate_example((q, c))  # Default numpy
tensor_data, tensor_labels = generate_example((q, c), type="tensor")
dataset = generate_example((q, c), type="dataset")
dataloader = generate_example((q, c), type="dataloader")


In [ ]:

# Test 2
save_dir = "C:/Datasets/Zhejiang20-23RS/temp_training"
save_dir=generate_large_dataset(q, m, n, t, c, save_dir)

from torch.utils.data import DataLoader
from tqdm import tqdm

# Generate or load datasets
train_dataset, test_dataset = generate_datasets(save_dir)
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print the length of the test dataset
print(len(test_dataset))

# Example usage of the test_loader
for x,y in tqdm(test_loader):
    # Process your training data here
    pass


# 0D

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from torch import nn
from torch.optim import Adam

# Generate synthetic data
np.random.seed(0)  # For reproducibility
x = np.random.uniform(-1, 1, (1000,10))  # 1000 samples
y = np.random.uniform(0, 3000, 1000)  # Target values in the range 0 to 3000

# Convert to PyTorch tensors
x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(-1)  # Adding an extra dimension for compatibility
y_tensor = torch.tensor(y, dtype=torch.float32)

# Create a PyTorch dataset and dataloader
dataset = TensorDataset(x_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=128, shuffle=True)

# Define the ANN model
class RegressionModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(10, 64),  # Input layer to first hidden layer
            nn.ReLU(),
            nn.Linear(64, 128),  # First hidden layer to second hidden layer
            nn.ReLU(),
            nn.Linear(128, 64),  # Second hidden layer to third hidden layer
            nn.ReLU(),
            nn.Linear(64, 1)  # Third hidden layer to output layer
        )
        self.criterion = nn.MSELoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y.unsqueeze(1))
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.0005)

# Define PyTorch Lightning trainer
trainer = pl.Trainer(
    max_epochs=5,
    callbacks=[EarlyStopping(monitor='train_loss', patience=50, min_delta=1e-4)]
)

# Initialize the model
model = RegressionModel()

model(x_tensor[:24])
# Train the model
trainer.fit(model, train_loader)


# 1D KNN,
C x 1 x 1 x 1


3. Make a regression model by using lightning.
TEST BY USING np.random.uniform(-1, 1,  )

Common setting	Looked-back steps	12
	Optimizer	Adam
	Loss function	MSE
	Learning rate	0.0005
	Epoch	100
	Early Stopping patience (delta)	50 (10-4)
	Batch size	128
  

Input is 1d.  batch x t x c, output is a float number (range from 0 to 3000) 
    Layer 3 Layer
	Kernel size	9
	Channel size	>9 
    for outher parameter make it by yourself according to your experience.
 x 1
knn, lgbm

In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.optim import Adam
from torch import nn
import numpy as np

class CNNRegressionModel(pl.LightningModule):
    def __init__(self):
        super(CNNRegressionModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=10, out_channels=16, kernel_size=9, padding=4)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=9, padding=4)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=9, padding=4)
        self.fc1 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = torch.mean(x, 2)  # Global Average Pooling
        x = self.fc1(x)
        return x
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.0005)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss)
        return loss

# Data preparation
np.random.seed(42)
x = np.random.uniform(-1, 1, (1000, 12, 10))  # 1000 samples, 10 input channels, 12 time steps
y = np.random.uniform(0, 3000, (1000, 1))  # Corresponding targets

x_tensor = torch.tensor(x, dtype=torch.float32).transpose(1, 2)  # Adjust dimensions for Conv1d (batch, channels, length)
y_tensor = torch.tensor(y, dtype=torch.float32)

dataset = TensorDataset(x_tensor, y_tensor)
train_size = int(len(dataset) * 0.8)
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)

# Initialize the PyTorch Lightning model
model = CNNRegressionModel()

# Trainer setup
trainer = pl.Trainer(
    max_epochs=10,
    callbacks=[pl.callbacks.EarlyStopping(monitor='train_loss', patience=50, min_delta=1e-4)],
    #progress_bar_refresh_rate=20
)

# Train the model
trainer.fit(model, train_loader, val_loader)


# 2D CNN
C x 5 x 5 x 1 

3. Make a regression model by using lightning.
TEST BY USING np.random.uniform(-1, 1,  )

Common setting	Looked-back steps	12
	Optimizer	Adam
	Loss function	MSE
	Learning rate	0.0005
	Epoch	100
	Early Stopping patience (delta)	50 (10-4)
	Batch size	128
 
 
Input is 2d.  batch x m x n x c, output is a float number (range from 0 to 3000) 
  
CNN	Number of convolution layers	4
	Kernel size	3 × 3
	Channel size	is a parameter to be initialized
	Activation function	Tanh

Then test it with 10 epochnh


In [3]:
import pytorch_lightning as pl
from torch import nn
import torch

class CNN2DRegression(pl.LightningModule):
    def __init__(self, input_channels, output_features=1):
        super().__init__()
        self.conv_net = nn.Sequential(
            nn.Conv2d(input_channels, 16, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.AdaptiveAvgPool2d(1)  # Global Average Pooling to reduce to 1x1x128
        )
        self.fc = nn.Linear(128, output_features)

    def forward(self, x):
        x=x.permute(0, 3, 1, 2)
        x = self.conv_net(x)
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch
        x = self.fc(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y.view(-1, 1))
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.0005)


In [6]:
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# Assuming m=28, n=28, c=3 for example
m, n, c = 28, 28, 3
x = np.random.uniform(-1, 1, (1000, m, n, c)).astype(np.float32)
y = np.random.uniform(0, 3000, (1000, 1)).astype(np.float32)

x_tensor = torch.tensor(x)
y_tensor = torch.tensor(y)

dataset = TensorDataset(x_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=128, shuffle=True)

from pytorch_lightning.callbacks import EarlyStopping

# Initialize the model
model = CNN2DRegression(input_channels=c)

# Trainer setup
trainer = pl.Trainer(
    max_epochs=10,
    callbacks=[EarlyStopping(monitor='train_loss', patience=50, min_delta=1e-4)],
    #progress_bar_refresh_rate=20,
    logger=False  # Disable logging for simplicity
)

# Train the model
trainer.fit(model, train_loader)
#model(x_tensor)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type       | Params
----------------------------------------
0 | conv_net | Sequential | 97.4 K
1 | fc       | Linear     | 129   
----------------------------------------
97.6 K    Trainable params
0         Non-trainable params
97.6 K    Total params
0.390     Total estimated model params size (MB)
C:\Users\isxzl\anaconda3\envs\EO\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 9: 100%|███████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 62.72it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|███████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 48.03it/s]


# 3D CNN
C x 5 x 5 x 1

3. Make a regression model by using lightning.
TEST BY USING np.random.uniform(-1, 1,  )

Common setting	Looked-back steps	12
	Optimizer	Adam
	Loss function	MSE
	Learning rate	0.0005
	Epoch	100
	Early Stopping patience (delta)	50 (10-4)
	Batch size	128


Input is 3d.  batch x m x n x t x c, output is a float number (range from 0 to 3000)
 


CNN	Number of convolution layers	4
	Kernel size	3 × 3 x 3
	Channel size	8 or 10
	Activation function	Tanh
 


In [8]:
import pytorch_lightning as pl
from torch import nn
import torch

class CNN3DRegression(pl.LightningModule):
    def __init__(self, input_channels, output_features=1):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv3d(input_channels, 8, kernel_size=(3, 3, 3), padding=(1, 1, 1)),
            nn.Tanh()
        )
        self.layer2 = nn.Sequential(
            nn.Conv3d(8, 16, kernel_size=(3, 3, 3), padding=(1, 1, 1)),
            nn.Tanh()
        )
        self.layer3 = nn.Sequential(
            nn.Conv3d(16, 32, kernel_size=(3, 3, 3), padding=(1, 1, 1)),
            nn.Tanh()
        )
        self.layer4 = nn.Sequential(
            nn.Conv3d(32, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1)),
            nn.Tanh()
        )
        self.global_avg_pool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Linear(64, output_features)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y.view(-1, 1))
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.0005)


In [9]:
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch

# Example dimensions
m, n, t, c = 8, 8, 12, 8  # Adjust dimensions as needed
x = np.random.uniform(-1, 1, (1000, c, t, m, n)).astype(np.float32)  # Synthetic data
y = np.random.uniform(0, 3000, (1000, 1)).astype(np.float32)  # Target output

x_tensor = torch.tensor(x)
y_tensor = torch.tensor(y)

dataset = TensorDataset(x_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=128, shuffle=True)


In [11]:
from pytorch_lightning.callbacks import EarlyStopping

model = CNN3DRegression(input_channels=c)
trainer = pl.Trainer(
    max_epochs=10,
    callbacks=[EarlyStopping(monitor='train_loss', patience=50, min_delta=1e-4)],
    #progress_bar_refresh_rate=20,
    logger=False  # To simplify output for this example
)

trainer.fit(model, train_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\isxzl\anaconda3\envs\EO\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:639: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoGeo\sample\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type              | Params
------------------------------------------------------
0 | layer1          | Sequential        | 1.7 K 
1 | layer2          | Sequential        | 3.5 K 
2 | layer3          | Sequential        | 13.9 K
3 | layer4          | Sequential        | 55.4 K
4 | global_avg_pool | AdaptiveAvgPool3d | 0     
5 | fc              | Linear            | 65    
------------------------------------------------------
74.5 K    Trainable params
0         Non-trainable params
74.5 K    Total params
0.298     Total estimated model params size (MB)


Epoch 35:  25%|██████████████████▌                                                       | 2/8 [00:00<00:00, 68.41it/s]

C:\Users\isxzl\anaconda3\envs\EO\lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


# 4D CNN
C x 5 x 5 x 5